2024-09-15 - Ronen Shekel

# General preparations

In [1]:
import matplotlib
matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
from lab.mplc.discrete_scan_result import DiscreetScanResult
from misc.misc import run_in_thread_simple, run_in_thread
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_modes2 import gen_input_spots_array, gen_output_modes_Unitary
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.mask_utils import remove_input_modes
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks, create_WFM_unitary_masks
from pianoq.misc.misc import detect_gaussian_spots_subpixel
import time 

C:\Users\owner\AppData\Local\Temp\ipykernel_4736\4151032235.py:1: DeprecationWarning: 'cgitb' is deprecated and slated for removal in Python 3.13
  from cgitb import small


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_16_fixing_phases_different_Us'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

# HW

In [3]:
# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

Got MPLC
Got Thorlabs motors
Got Zaber motors
Got Time tagger


In [4]:
def get_masks(U_no):
    paths = glob.glob(fr'{dir_path}\U{U_no}U*.masks')
    assert len(paths) == 1, 'should be only one '
    msks = MPLCMasks()
    msks.loadfrom(paths[0])
    return msks.real_masks
    
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
U_nos = np.arange(5, 10)

# Create masks WFM

In [5]:
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_QKD_masks, create_WFM_unitary_masks
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_10_09_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
mplc_sims = []
for U_no in U_nos:
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    masks_path =    fr'{dir_path}\U{U_no}U_{timestamp}.masks'
    mplc_res_path = fr'{dir_path}\U{U_no}U_{timestamp}.mplcsim'
    mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path)
    mplc_sim.res.saveto(mplc_res_path, smaller=True)
    mplc_sims.append(mplc_sim)

100%|██████████| 15/15 [03:00<00:00, 12.07s/it]


In [6]:
for U_no in U_nos:
    mplc_sims[U_no].res._calc_normalized_overlap()
    print("Are phases close to 0?")
    display(np.angle(np.diag(mplc_sims[U_no].res.forward_overlap)))
    print('Are overlaps ~.65?')
    display(np.angle(np.abs(mplc_sims[U_no].res.forward_overlap)))
    mplc_sims[U_no].res._calc_fidelity()
    print('Is fidelity ~.98?')
    display(mplc_sims[U_no].res.fidelity)

IndexError: list index out of range

In [8]:
for U_no in U_nos:
    mplc_sims[U_no].res.show_overlap()

In [12]:
for U_no in U_nos:
    fig, axes = plt.subplots(2, 5)
    for i, ax in enumerate(axes.flat):
        # ax.imshow(np.angle(res.masks[i][360:720, 140:280]), cmap='gray')
        ax.imshow(np.angle(mplc_sims[U_no].res.masks[i][360:720, 140:280]), cmap='gray')
        fig.show()

In [15]:
mplc_sims[3].res.show_all(4)

# Find locations with single counts 

In [ ]:
resolution = 1

scanner_sig, _, _, _ = get_signal_scanner(integration_time=1.0, coin_window=2e-9, resolution=resolution, half_scan=True, no_hw_mode=True)
scanner_idl, _, _, _ = get_idler_scanner(integration_time=1.0, coin_window=2e-9, resolution=resolution, half_scan=True, no_hw_mode=True)

for U_no in U_nos:
    print(f"--- Single counts scan #{U_no} ---")
    masks = get_masks(U_no)
    masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
    mplc.load_masks(masks, linear_tilts=True)
    mplc.restore_location()
    
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    singles_idler_path = fr'{dir_path}\U{U_no}U_{timestamp}_idler_res_{resolution}.scan'
    singles_signal_path = fr'{dir_path}\U{U_no}U_{timestamp}_signal_res_{resolution}.scan'
    
    scanner_sig.saveto_path = singles_signal_path
    scanner_idl.saveto_path = singles_idler_path
    t = run_in_thread(scanner_sig.scan, x_motor=mxs, y_motor=mys, ph=tt)
    t2 = run_in_thread(scanner_idl.scan, x_motor=mxi, y_motor=myi, ph=tt)    
    t.join()
    t2.join()

--- Single counts scan #5 ---


Traceback (most recent call last):
  File "C:\code\pianoq_results\pianoq_results\scan_result.py", line 183, in saveto
    f = open(path, 'wb')
    
FileNotFoundError: [Errno 2] No such file or directory: 'G:\\My Drive\\Projects\\MPLC\\results\\lab\\2024_09_16_fixing_phases_different_Us\\U5U_2024_09_16_18_41_09_signal_res_1.scan'


## mark spots locations  

In [7]:
from pianoq_results.scan_result import ScanResult

for U_no in U_nos:
    path_sig = glob.glob(fr'{dir_path}\U{U_no}U*signal_res_1.scan')[0]
    path_idl = glob.glob(fr'{dir_path}\U{U_no}U*idler_res_1.scan')[0]
    
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    locs_signal_path = fr'{dir_path}\U{U_no}U_{timestamp}_s2_sig.locs'
    locs_idler_path = fr'{dir_path}\U{U_no}U_{timestamp}_s1_idl.locs'
    
    res_sig = ScanResult()
    res_idl = ScanResult()
    res_idl.loadfrom(path_idl)
    res_sig.loadfrom(path_sig)
    # res_idl.show_singles()
    # res_sig.show_singles()
    
    sig_locs = detect_gaussian_spots_subpixel(res_sig.single2s, res_sig.X, res_sig.Y, sort_top_to_bottom=True)
    idl_locs = detect_gaussian_spots_subpixel(res_idl.single1s, res_idl.X, res_idl.Y, sort_top_to_bottom=False)
    
    f = open(locs_signal_path, 'wb')
    np.savez(f, locs=sig_locs)
    f.close()

    f = open(locs_idler_path, 'wb')
    np.savez(f, locs=idl_locs)
    f.close()
    
    # _ = res_sig.get_xys(1, saveto_dir=dir_path)
    # _ = res_idl.get_xys(2, saveto_dir=dir_path)

# Find phases

In [18]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder
import glob 
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan
import numpy as np
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
import time 


timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_path = fr'{dir_path}\{timestamp}.phases'
initiail_phases = np.zeros(25 * 2)
integration_time = 20
pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=initiail_phases, coin_window=2e-9, no_hw_mode=True)

pf.zaber_ms = zaber_ms
pf.m_sig_x = mxs
pf.m_sig_y = mys
pf.m_idl_x = mxi
pf.m_idl_y = myi
pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

for U_no in U_nos:
    # zeroing object 
    timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
    phases_path = fr'{dir_path}\U{U_no}U_{timestamp}.phases'
    pf.res.path = phases_path
    initiail_phases = np.zeros(25 * 2)
    pf.res.initial_phases = initiail_phases
    pf.res.phases = initiail_phases
    
    # masks 
    masks = get_masks(U_no)
    masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
    mplc.load_masks(masks, linear_tilts=True)
    initiail_phases = np.zeros(25 * 2)
    pf.orig_masks = mplc.masks.copy()
    
    # locs 
    locs_sig_path = glob.glob(fr'{dir_path}\U{U_no}U*sig.locs')[0]
    locs_idl_path = glob.glob(fr'{dir_path}\U{U_no}U*idl.locs')[0]
    locs_sig = np.load(locs_sig_path)['locs']
    locs_idl = np.load(locs_idl_path)['locs']
    
    # i,j of corr matrix that is supposed to be strong 
    i = 2
    j = 2
    pf.m_idl_x.move_absolute(locs_idl[i, 0])
    pf.m_idl_y.move_absolute(locs_idl[i, 1])
    pf.m_sig_x.move_absolute(locs_sig[j, 0])
    pf.m_sig_y.move_absolute(locs_sig[j, 1])        
    mplc.restore_location()
    pf.find_phases()

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
3,0
3,1
3,2
3,3
3,4
3,5
3,6
3,7
3,8
3,9
4,0
4,1
4,2
4,3
4,4
4,5
4,6
4,7
4,8
4,9
5,0
5,1
5,2
5,3
5,4
5,5
5,6
5,7
5,8
5,9
6,0
6,1
6,2
6,3
6,4
6,5
6,6
6,7
6,8
6,9
7,0
7,1
7,2
7,3
7,4
7,5
7,6
7,7
7,8
7,9
8,0
8,1
8,2
8,3
8,4
8,5
8,6
8,7
8,8
8,9
9,0
9,1
9,2
9,3
9,4
9,5
9,6
9,7
9,8
9,9
0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
3,0
3,1
3,2
3,3
3,4
3,5
3,6
3,7
3,8
3,9
4,0
4,1
4,2
4,3
4,4
4,5
4,6
4,7
4,8
4,9
5,0
5,1
5,2
5,3
5,4
5,5
5,6
5,7
5,8
5,9
6,0
6,1
6,2
6,3
6,4
6,5
6,6
6,7
6,8
6,9
7,0
7,1
7,2
7,3
7,4
7,5
7,6
7,7
7,8
7,9
8,0
8,1
8,2
8,3
8,4
8,5
8,6
8,7
8,8
8,9
9,0
9,1
9,2
9,3
9,4
9,5
9,6
9,7
9,8
9,9
0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
3,0
3,1
3,2
3,3
3,4
3,5
3,6
3,7
3,8
3,9
4,0
4,1
4,2
4,3
4,4
4,5
4,6
4,7
4,8
4,9


In [23]:
fig, ax = plt.subplots()
for U_no in U_nos:
    path = glob.glob(fr'{dir_path}\U{U_no}U*2024_09_15*.phases')[0]
    res = PhaseFinderResult()
    res.loadfrom(path)
    ax.plot(res.phases[modes_to_keep - 1], label=U_no)
    # ax.imshow(res.coincidences, label=U_no)
ax.legend()
fig.show()

In [24]:
U_no = 1
path = glob.glob(fr'{dir_path}\U{U_no}U*2024_09_15*.phases')[0]
res = PhaseFinderResult()
res.loadfrom(path)

fig, ax = plt.subplots()
ax.imshow(res.coincidences, label=U_no)
fig.show()

# Scan correlations 

In [28]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.mplc_device import MPLCDevice
import glob
import time 

backlash = 0.0
wait_after_move = 0.3
integration_time = 40
coin_window = 1.2e-9
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

scan_path = fr'{dir_path}\{timestamp}.dscan'
locs_sig = None 
locs_idl = None
dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window, no_hw_mode=True)
dps.zaber_ms = zaber_ms
dps.m_sig_x = mxs
dps.m_sig_y = mys
dps.m_idl_x = mxi
dps.m_idl_y = myi
dps.time_tagger = tt
dps.time_tagger.set_integration_time(integration_time)
time.sleep(1)


for U_no in U_nos:
    # For each of these 5 Us, use all 5 options of phases (U_i+phases_i should of course be the best) 
    for phases_no in range(5):
        # phases 
        # phases_no = U_no  # use the found good phases. 
        
        phases_result = PhaseFinderResult()
        phases_path = glob.glob(fr'{dir_path}\U{phases_no}U*2024_09_15*.phases')[0]
        phases_result.loadfrom(phases_path)
        
        # masks 
        masks = get_masks(U_no)
        masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
        masks = add_phase_input_spots(masks, phases_result.phases)
        mplc.load_masks(masks, linear_tilts=True)
        
        # locs 
        locs_sig_path = glob.glob(fr'{dir_path}\U{U_no}U*sig.locs')[0]
        locs_idl_path = glob.glob(fr'{dir_path}\U{U_no}U*idl.locs')[0]
        locs_sig = np.load(locs_sig_path)['locs']
        locs_idl = np.load(locs_idl_path)['locs']
    
        timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
        scan_path = fr'{dir_path}\U{U_no}U_{timestamp}_Haar_phases_H{phases_no}H_exp_{integration_time}s_coin_{int(coin_window*1e12)}ps.dscan'
        dps.res.path = scan_path
        dps.res.locs_idler = locs_idl
        dps.res.locs_signal = locs_sig
        time.sleep(1)
        mplc.restore_location()
        time.sleep(1)
        dps.scan()


beginning scan
0, 0: 6014.25, 5100.23, 2.80
0, 1: 5979.53, 6118.30, 0.18
0, 2: 6009.28, 6216.75, 0.20
0, 3: 6008.40, 7140.18, 0.35
0, 4: 6007.98, 8281.12, 0.55
1, 0: 8085.03, 5093.98, 0.73
1, 1: 8075.93, 6107.58, 4.20
1, 2: 8057.70, 6242.90, 0.55
1, 3: 8079.55, 7182.28, 1.15
1, 4: 8099.55, 8256.75, 0.50
2, 0: 6776.30, 5126.90, 0.33
2, 1: 6763.88, 6118.03, 0.45
2, 2: 6765.88, 6253.10, 3.70
2, 3: 6776.55, 7185.50, 0.45
2, 4: 6793.25, 8257.40, 0.45
3, 0: 8449.70, 5106.85, 0.25
3, 1: 8457.83, 6152.90, 2.00
3, 2: 8501.95, 6258.12, 0.73
3, 3: 8454.02, 7206.03, 5.30
3, 4: 8421.75, 8300.98, 0.62
4, 0: 9428.10, 5123.70, 0.30
4, 1: 9456.98, 6151.43, 0.58
4, 2: 9487.45, 6296.75, 0.73
4, 3: 9460.85, 7219.75, 0.45
4, 4: 9481.62, 8304.58, 8.35
beginning scan
0, 0: 6274.73, 5575.12, 2.43
0, 1: 6281.83, 7583.20, 0.35
0, 2: 6300.35, 6293.30, 0.38
0, 3: 6292.75, 7977.45, 0.33
0, 4: 6285.23, 8825.08, 1.43
1, 0: 8305.40, 5601.53, 0.43
1, 1: 8342.40, 7621.30, 8.83
1, 2: 8295.65, 6325.95, 0.43
1, 3: 8321.48

In [27]:
from pianoq.lab.mplc.discrete_scan_result import DiscreetScanResult
for U_no in U_nos:
    path = glob.glob(rf'{dir_path}\U{U_no}U*2024_09_15*.dscan')[1]
    s = DiscreetScanResult()
    s.loadfrom(path)
    s.show()